In [169]:
import urllib2, os, sys, subprocess
import googleapiclient.discovery
import os, sys
import random
import time 
import paramiko
import tinydb
import datetime

In [145]:
_d = tinydb.TinyDB('test.json')
q = tinydb.Query()
_d.search(q.vmname=='test')

[]

In [146]:
_d.update({'t_preempted':423}, q.vmname=='test')

[]

In [159]:
dt = datetime.datetime.now().isoformat()

In [162]:
res = _d.search(q.vmname=='test')

In [167]:
len(res)

0

In [85]:
def read_instance_properties():
    #compute-nodes: list of other instances that belong to cluster. Try using series-[1-100] notation
    #name of slurm-master 
    #machine type, can we get that from /proc/cpuinfo? 
    pass

In [97]:
#Globals that are used in most GCP functions 
project='first-220321'
zone='us-east1-b'
vmnum=0
compute = googleapiclient.discovery.build('compute', 'v1')

In [168]:
def get_jobdb(self):
    return tinydb.TinyDB('jobdb.json')

def get_vmdb(self):
    return tinydb.TinyDB('vmdb.json')


In [96]:
def num_cpus():
    """ Number of CPUs if we are running this script locally"""
    try:
        np=subprocess.check_output('cat /proc/cpuinfo | grep processor | wc -l', shell=True)
        p = int(np)
        return p
    except:
        p = 1 
        return p 

In [87]:
def machine_type(m):
    #Cat proc/cpuinfo, and some memory ? 
    #cpus = num_cpus()
    cpus = int(m.split('-')[-1])
    machine = {'sockets': 1, 'cores': cpus/2, 'threads': 2, 'memory': 1000}
    return machine 
#TODO: Convert to string type so that we can use it? 
    

In [4]:
slurm_master="ubslurm1"
namegrp="absynth"
num_nodes=4
mtype='n1-highcpu-16'
master_exists=True 
compute_nodes="absynth[1-10]"

In [98]:
def start_instance(mtype, zone, name, startupscriptstr):
    """ Hidden inputs: VM-image """
    
    machine_type = "zones/{}/machineTypes/{}".format(zone, mtype)
    
    instance_body={
        'name':name,
        'machineType':machine_type, 
        'scheduling':
          {
        'preemptible': 'true'
          },
        'disks': [
        {
            'boot': True,
            'autoDelete': True,
            'initializeParams': {
                'sourceImage': "global/images/ubs2"
                }
        }],
        'metadata' : {
            "items" : [
                {
                    "key": "startup-script",
                    "value":startupscriptstr
                }
            ]
        },
        'networkInterfaces': [{
        'network': 'global/networks/default',
        'accessConfigs': [
            {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
        ]
    }],
    }
    print(str(instance_body))
    
    response = compute.instances().insert(project=project,zone=zone,body=instance_body).execute()
    return response 
    

In [24]:
#machine={'sockets': 1, 'cores': 1, 'threads': 2, 'memory': 1000}

In [126]:
def generate_slurm_config(slurm_master, machine, compute_nodes):
    slurmconfstr = """
# slurm.conf file generated by configurator easy.html.
# Put this file on all nodes of your cluster.
# See the slurm.conf man page for more information.
#
ControlMachine={slurm_master}
AuthType=auth/none
#CheckpointType=checkpoint/none
#CryptoType=crypto/munge
#ControlAddr=
#
#MailProg=/bin/mail
MpiDefault=none
#MpiParams=ports=#-#
ProctrackType=proctrack/pgid
ReturnToService=1
SlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid
#SlurmctldPort=6817
SlurmdPidFile=/var/run/slurm-llnl/slurmd.pid
#SlurmdPort=6818
SlurmdSpoolDir=/var/lib/slurm-llnl/slurmd
#SlurmUser=slurm
SlurmdUser=root
StateSaveLocation=/var/lib/slurm-llnl/slurmctld
SwitchType=switch/none
TaskPlugin=task/none
JobCredentialPrivateKey=/home/prateek3_14/slurmkey
#
JobCompType=jobcomp/filetxt
JobCompLoc=/var/log/slurmjobs
DebugFlags=NO_CONF_HASH
# TIMERS
#KillWait=30
#MinJobAge=300
#SlurmctldTimeout=120
#SlurmdTimeout=300
#
#
# SCHEDULING
FastSchedule=1
SchedulerType=sched/builtin
#SchedulerPort=7321
SelectType=select/linear
#
#
# LOGGING AND ACCOUNTING
AccountingStorageType=accounting_storage/none
ClusterName=ubslurm1
#JobAcctGatherFrequency=30
JobAcctGatherType=jobacct_gather/none
#SlurmctldDebug=3
SlurmctldLogFile=/var/log/slurm-llnl/slurmctld.log
#SlurmdDebug=3
SlurmdLogFile=/var/log/slurm-llnl/slurmd.log
#
#
# COMPUTE NODES \n""".format(slurm_master=slurm_master)

    slurmconfstr += ' '.join(("NodeName=DEFAULT",
                          "Sockets="        + str(machine['sockets']),
                          "CoresPerSocket=" + str(machine['cores']),
                          "ThreadsPerCore=" + str(machine['threads']),
                          "State=UNKNOWN"))

    slurmconfstr += '\n'
    #slurmconfstr += 'NodeName={slurm_master} \n'.format(slurm_master=slurm_master)
    slurmconfstr += 'NodeName={compute_nodes} \n'.format(compute_nodes=compute_nodes)


    slurmconfstr += "\n PartitionName=long Nodes={compute_nodes} Default=YES MaxTime=INFINITE State=UP \n".format(compute_nodes=compute_nodes)
    return slurmconfstr

In [26]:
#print slurmconfstr

In [127]:
def get_startup_script(slurmconfstr):
    """ Copy slurm config, and start slurmd. TODO: if master, then slurmctld"""
    
    startupscriptstr = """
#!/bin/bash

logger "Startup Script Begins.... "
logger "Running as `whoami`" 

systemctl stop slurmd 
systemctl stop slurmctld 

echo > /etc/slurm-llnl/slurm.conf

cat <<\EOF >> /etc/slurm-llnl/slurm.conf
{slurmconfstr}
EOF

systemctl start slurmd 

logger "Slurm conf applied, startup script ending" 

exit 0

""".format(slurmconfstr=slurmconfstr)
    return startupscriptstr

In [128]:
def get_instance_ip(instance):
    #Let project and zone be global! 
    return ins.get(project=project, zone=zone, instance=instance).execute().\
    get('networkInterfaces')[0].get('accessConfigs')[0].get('natIP')


In [129]:
def gcp_ssh(instance):
    instance_ip = get_instance_ip(instance)
    #
    client=paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    #Copy the new slurm config file 
    client.connect(instance_ip, username='prateek3_14',key_filename='/home/prateeks/.ssh/gce')
    return client 

In [136]:
#scp the file first into /tmp/reconf_script.sh 
#chmod +x reconf_script.sh 
#execute script 
def reconfig_master(master, slurmconf_str):
    
    client = gcp_ssh(master)
    
    i,o,e=client.exec_command("cat > /tmp/slurmconf")
    i.write(slurmconf_str)
    i.close()
    client.close() 
    #TODO: Can we reuse the same connection? 
    
    client = gcp_ssh(master)
    i,o,e=client.exec_command("sudo slurm_reload_cfg /tmp/slurmconf")
    client.close()
    #Above script copies new cfg and restarts slurm daemons 
    print("Master reconfigured")
    #TODO: Set up strigger node down for each launched node? 
    return 


In [88]:
#Node down.
#Two ways to detect via strigger
#Either set a trigger for each node 
#Or, --set --jobid --down --program foo.sh  , which triggers if any node is down and nodeid is passed as arg


In [11]:
compute = googleapiclient.discovery.build('compute', 'v1')

In [12]:
compute.instances().list(project=project,zone=ze).execute()

{u'id': u'projects/first-220321/zones/us-east1-b/instances',
 u'items': [{u'cpuPlatform': u'Unknown CPU Platform',
   u'creationTimestamp': u'2019-02-18T11:37:09.182-08:00',
   u'deletionProtection': False,
   u'disks': [{u'autoDelete': True,
     u'boot': True,
     u'deviceName': u'persistent-disk-0',
     u'guestOsFeatures': [{u'type': u'VIRTIO_SCSI_MULTIQUEUE'}],
     u'index': 0,
     u'interface': u'SCSI',
     u'kind': u'compute#attachedDisk',
     u'licenses': [u'https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1804-lts'],
     u'mode': u'READ_WRITE',
     u'source': u'https://www.googleapis.com/compute/v1/projects/first-220321/zones/us-east1-b/disks/aardvark15',
     u'type': u'PERSISTENT'}],
   u'id': u'2048871841028028299',
   u'kind': u'compute#instance',
   u'labelFingerprint': u'42WmSpB8rSM=',
   u'machineType': u'https://www.googleapis.com/compute/v1/projects/first-220321/zones/us-east1-b/machineTypes/n1-standard-1',
   u'metadata': {

In [170]:
def launch_cluster(namegrp, num_nodes, mtype, start_id=1, slurm_master=None):
    machine = machine_type(mtype)
    
    compute_nodes = namegrp+"[1-"+str(num_nodes*3)+"]"
    #*3 is just a margin for safety in case we fail and have to rerun the job without reconfiguring master 
    configstr = generate_slurm_config(slurm_master, machine, compute_nodes)
    cnodes_launched = []
    if slurm_master is None:
        print("Master must be running, not supported yet, exiting")
        return
    
    reconfig_master(slurm_master, configstr)
    
    launch_workers = True 
    
    if launch_workers is False:
        return 
    
    for i in range(start_id, num_nodes+start_id): 
        name = namegrp+str(i)
        
        start_instance(mtype, zone, name, get_startup_script(configstr))
        cnodes_launched.append(name)
        #Add to the vmdb 
        vmdb = get_vmdb()
        t_start = datetime.datetime.now().isoformat()
        vmdb.insert({'vmname':name, 't_start':t_start, 'type':mtype, 'status':'running'})
        vmdb.close()
        
        time.sleep(5)
    print("Cluster Launched")
    return (slurm_master, cnodes_launched)

In [132]:
launch_cluster('absynth', 2, 'n1-highcpu-16', 5, 'ubslurm1')

Master reconfigured
{'machineType': 'zones/us-east1-b/machineTypes/n1-highcpu-16', 'name': 'absynth5', 'disks': [{'initializeParams': {'sourceImage': 'global/images/ubs2'}, 'boot': True, 'autoDelete': True}], 'scheduling': {'preemptible': 'true'}, 'metadata': {'items': [{'value': '\n#!/bin/bash\n\nlogger "Startup Script Begins.... "\nlogger "Running as `whoami`" \n\nsystemctl stop slurmd \nsystemctl stop slurmctld \n\necho > /etc/slurm-llnl/slurm.conf\n\ncat <<\\EOF >> /etc/slurm-llnl/slurm.conf\n\n# slurm.conf file generated by configurator easy.html.\n# Put this file on all nodes of your cluster.\n# See the slurm.conf man page for more information.\n#\nControlMachine=ubslurm1\nAuthType=auth/none\n#CheckpointType=checkpoint/none\n#CryptoType=crypto/munge\n#ControlAddr=\n#\n#MailProg=/bin/mail\nMpiDefault=none\n#MpiParams=ports=#-#\nProctrackType=proctrack/pgid\nReturnToService=1\nSlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid\n#SlurmctldPort=6817\nSlurmdPidFile=/var/run/slurm-llnl

('ubslurm1', ['absynth5', 'absynth6'])

In [171]:
def run_job(master, num_nodes, machine):
    
    cores = machine['cores']
    runfile = "/home/prateek3_14/sb_confinement.sh"
    simparams = ""
    sbatcmd = "sbatch --parsable -N {num_nodes} -c {cores} -n {num_nodes} {runfile} {simparams}".format(\
        num_nodes=num_nodes, cores=cores, runfile=runfile, simparams=simparams)
    #Just returns the job-id 
    
    sshclient = gcp_ssh(master)
    i,o,e = sshclient.exec_command(sbatcmd)
    jobid = o.read() 
    o.close()
    sshclient.close() 
    
    #TODO strigger 
    #strigger --set --fini --program /home/prateek3_14/scispot/code/jobfin.sh --jobid 75
    #strigger --jobid=1234 --down --program=/home/prateek3_14/scispot/code/jobfail.sh 
    
    #TODO: Update state of the job and add to job dict with the time we launched 
    #Metadata for a job: jobid, launchtime, sbatch config, simparams, codepath,.. 
    #starttime = time.get()
    jobdb = get_jobdb() 
    t_start = datetime.datetime.now().isoformat()
    jobdb.insert({'jobname':jobid, 't_start':t_start, 'resources':(nodes, cores), 'state':'running'})
    jobdb.close()
    return jobid


In [121]:
run_job('ubslurm1', 4, machine_type('n1-highcpu-16'))

'75\n'

In [138]:
db=tinydb.TinyDB('testdb.json')

In [139]:
db.insert({'foo':1})

1

In [140]:
db.all()

[{u'foo': 1}]

In [143]:
q = tinydb.Query()
db.search(q.fo2o == 2)

[]

In [47]:
client=paramiko.SSHClient()

In [48]:
client.load_system_host_keys()

In [49]:
hname='34.73.2.158'

In [50]:
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hname, username='prateek3_14',key_filename='/home/prateeks/.ssh/gce')

In [51]:
i,o,e=client.exec_command("cat > t")

In [52]:
i.write("Hello world!\n \n")

In [53]:
i.close()

In [35]:
o.read()
client.close()

In [19]:
ins=compute.instances()

In [79]:
instance='ubslurm1'
ins.get(project=project, zone=zone, instance=instance).execute().get('networkInterfaces')[0].get('accessConfigs')[0].get('natIP')

u'34.73.2.158'

In [80]:
get_instance_ip('ubslurm1')

u'34.73.2.158'

In [ ]:
prepare_master('ubslurm1', )